# Data Importer

I have not used this first dataset that I have downloaded but the main dataset I am basing on is using this dataset, so, to give credit and show the root dataset, I am importing this here.

In [2]:
import pandas as pd

dismissal = pd.read_csv('./Data/ceo_dismissal_data.csv')

dismissal.head(5)

,dismissal_dataset_id,coname,gvkey,year,co_per_rol,exec_fullname,Departure Code,ceo_dismissal,Interim & Co-CEO,tenure_no,max_tenure_ceodb,The fiscal year during which the CEO exited - for clarity,leftofc,Still There,Notes and Thoughts,Notes 2
0,1,AAR CORP,1004,1995,5622,Ira A. Eichner,5.0,0.0,NaN,1,1,1996.0,10/1/1996,NaN,Mr. Ira A. Eichner served as CEO until October...,https://www.comparably.com/companies/aar-corp/...
1,2,AAR CORP,1004,2017,5623,David P. Storch,5.0,0.0,NaN,1,1,2018.0,5/31/2018,NaN,Agreed to stay as non-executive chairman post ...,https://advance-lexis-com.umiss.idm.oclc.org/a...
2,3,AAR CORP,1004,2018,51547,"John McClain Holmes, III",NaN,NaN,NaN,1,1,NaN,NaN,17nov2020,NaN,https://www.aarcorp.com/about/corporate-officers/
3,4,ADC TELECOMMUNICATIONS INC,1013,2000,2611,William J. Cadogan,6.0,0.0,NaN,1,1,2001.0,2/15/2001,NaN,"William Cadogan, the retiring CEO and chairman...",https://www.bizjournals.com/twincities/stories...
4,5,ADC TELECOMMUNICATIONS INC,1013,2003,23275,Richard R. Roscitt,6.0,0.0,NaN,1,1,2003.0,8/13/2003,NaN,"Richard R. ""Rick"" Roscitt has accepted a new p...",https://www.businesswire.com/news/home/2003081...


## Main Dataset

This dataset is taken from Giorgi Kukishvili from this [address](https://github.com/gkukish/CEO-Dismissal-Causal-Inference/tree/main) since it provides a good basis for main variables to analyze and infer from.

In [3]:
wfinancial = pd.read_csv("./Data/final_dataset_publishable.csv")

wfinancial.head(5)

,CRSP Ticker,Fiscal Quarter,Net Income,Operating Income,EPS Diluted,EPS Basic,CEO Dismissal,10 Year Treasury Yield,3 Month Treasury Yield,3 Month Treasury Yield (Bond Equivalent Basis),Spread,Recession Probability,Recession,Adjusted Close,GICS Sector,GICS Sub-Industry,Date added
0,ABT,2008Q1,NaN,0.138628,NaN,NaN,0,3.663333,2.043333,2.083440,1.579894,0.398800,1.0,18.284336,Health Care,Health Care Equipment,1957-03-04
1,ABT,2008Q2,0.192314,0.180749,0.85,0.86,0,3.886667,1.626667,1.656223,2.230444,0.303028,1.0,17.679647,Health Care,Health Care Equipment,1957-03-04
2,ABT,2008Q3,0.186259,0.144658,0.69,0.70,0,3.863333,1.493333,1.519986,2.343348,0.232969,1.0,19.339706,Health Care,Health Care Equipment,1957-03-04
3,ABT,2008Q4,NaN,0.193229,NaN,NaN,0,3.253333,0.296667,0.301203,2.952131,0.152300,1.0,18.055019,Health Care,Health Care Equipment,1957-03-04
4,ABT,2009Q1,0.157889,0.214128,0.92,0.93,0,2.736667,0.213333,0.216425,2.520241,0.073081,1.0,16.253757,Health Care,Health Care Equipment,1957-03-04


## Adding Some More With YahooFinance

Needing a bit more granular detail, I wanted to bring "beta" as a measure of volatility, "average closing price" to track performance, "dividend yield" to assess possible issues with dividend returns.

In [4]:
import yfinance as yf
import pandas as pd

# Function to fetch historical stock prices for the fiscal quarter
def fetch_yahoo_financial_data(row):
    # Extract ticker and fiscal quarter
    ticker = row['CRSP Ticker']
    fiscal_quarter = row['Fiscal Quarter']

    # Parse the fiscal quarter into start and end dates
    year, quarter = fiscal_quarter.split('Q')
    year = int(year)
    quarter = int(quarter)

    # Map quarter to start and end months
    quarter_start_months = {1: '01', 2: '04', 3: '07', 4: '10'}
    quarter_end_months = {1: '03', 2: '06', 3: '09', 4: '12'}

    start_date = f"{year}-{quarter_start_months[quarter]}-01"
    end_date = f"{year}-{quarter_end_months[quarter]}-31"  # Last day of the quarter

    # Initialize yfinance for the given ticker
    stock = yf.Ticker(ticker)

    # Fetch historical stock prices for the quarter
    try:
        historical_prices = stock.history(start=start_date, end=end_date)
        # Calculate metrics based on the historical prices
        closing_price_mean = historical_prices['Close'].mean()
        dividend_yield = historical_prices['Dividends'].sum() / closing_price_mean if closing_price_mean else None
    except:
        closing_price_mean = None
        dividend_yield = None

    # Fetch current company info (for beta, etc.)
    try:
        company_info = stock.info
    except:
        company_info = {}

    # Extract relevant financial metrics (Beta)
    beta = company_info.get('beta', None)

    # Return all values in a dictionary
    return pd.Series({
        'Beta': beta,
        'Average Closing Price': closing_price_mean,
        'Dividend Yield': dividend_yield
    })

# Apply the function to each row in your wfinancial dataframe
wfinancial[['Beta', 'Average Closing Price', 'Dividend Yield']] = wfinancial.apply(fetch_yahoo_financial_data, axis=1)

# Display the updated dataframe
print(wfinancial.head())



$BTU: possibly delisted; no price data found  (1d 2008-10-01 -> 2008-12-31) (Yahoo error = "Data doesn't exist for startDate = 1222833600, endDate = 1230699600")
$BTU: possibly delisted; no price data found  (1d 2009-01-01 -> 2009-03-31) (Yahoo error = "Data doesn't exist for startDate = 1230786000, endDate = 1238472000")
$BTU: possibly delisted; no price data found  (1d 2009-10-01 -> 2009-12-31) (Yahoo error = "Data doesn't exist for startDate = 1254369600, endDate = 1262235600")
$BTU: possibly delisted; no price data found  (1d 2010-01-01 -> 2010-03-31) (Yahoo error = "Data doesn't exist for startDate = 1262322000, endDate = 1270008000")
$BTU: possibly delisted; no price data found  (1d 2010-10-01 -> 2010-12-31) (Yahoo error = "Data doesn't exist for startDate = 1285905600, endDate = 1293771600")
$BTU: possibly delisted; no price data found  (1d 2011-01-01 -> 2011-03-31) (Yahoo error = "Data doesn't exist for startDate = 1293858000, endDate = 1301544000")
$BTU: possibly delisted; no 

  CRSP Ticker Fiscal Quarter  Net Income  Operating Income  EPS Diluted  \
0         ABT         2008Q1         NaN          0.138628          NaN   
1         ABT         2008Q2    0.192314          0.180749         0.85   
2         ABT         2008Q3    0.186259          0.144658         0.69   
3         ABT         2008Q4         NaN          0.193229          NaN   
4         ABT         2009Q1    0.157889          0.214128         0.92   

   EPS Basic  CEO Dismissal  10 Year Treasury Yield  3 Month Treasury Yield  \
0        NaN              0                3.663333                2.043333   
1       0.86              0                3.886667                1.626667   
2       0.70              0                3.863333                1.493333   
3        NaN              0                3.253333                0.296667   
4       0.93              0                2.736667                0.213333   

   3 Month Treasury Yield (Bond Equivalent Basis)    Spread  \
0          

In [5]:
import os

# Define the directory path for the 'Data' folder
output_dir = 'Data'

# Check if the folder exists, and if not, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define the file path for the CSV export
output_file = os.path.join(output_dir, 'wfinancial_updated.csv')

# Export the dataframe to CSV inside the 'Data' folder
wfinancial.to_csv(output_file, index=False)  # index=False to exclude the row numbers

# Print the output path to confirm
print(f"DataFrame exported to: {output_file}")


DataFrame exported to: Data/wfinancial_updated.csv


## Adding Macroeconomic Data

Using the data from FRED, we can also add the macroeconomic data to the existing dataframe to understand the general market conditions. In the following cells, the following variables have been added:


- Unemployment Rate (UNRATE): The percentage of the labor force that is unemployed and actively seeking work, indicating the health of the labor market.
- GDP Growth Rate (A191RL1Q225SBEA): The quarterly percentage change in real GDP, showing the overall economic growth or contraction.
- Consumer Price Index (CPI) (CPIAUCSL): A measure of inflation based on the average price change of goods and services purchased by urban consumers.
- Federal Funds Rate (FEDFUNDS): The interest rate at which banks lend to each other overnight, influencing borrowing costs across the economy.
- Consumer Sentiment Index (UMCSENT): Reflects consumer confidence regarding the economy, influencing spending behavior.
- Private Nonresidential Fixed Investment (PNFIC1): Tracks business investments in structures and equipment, indicating business confidence and economic activity.
- Labor Force Participation Rate (CIVPART): The percentage of the working-age population that is either employed or actively seeking employment.
- Industrial Production Index (INDPRO): Measures the output of industrial sectors like manufacturing and mining, indicating industrial economic performance.
- Housing Starts (HOUST): The number of new residential construction projects started, signaling trends in the housing market and construction industry.
- Corporate Profits (CP): Measures after-tax corporate profits across the economy, reflecting business profitability.
- Disposable Personal Income (DSPIC96): Adjusted for inflation, this represents the income available to individuals after taxes, indicating purchasing power.


In [28]:
from fredapi import Fred
import pandas as pd

def load_api_key(file_path='creds.txt'):
    with open(file_path, 'r') as file:
        # Read each line and extract the key-value pairs
        for line in file:
            if line.startswith("FRED_API_KEY"):
                return line.strip().split('=')[1] 

# Initialize FRED API with your key
fred = Fred(api_key=load_api_key())  # Replace with your FRED API key

# Cache for storing fetched data by fiscal quarter
cache = {}

# Function to fetch macroeconomic data for a given fiscal quarter (with caching)
def fetch_fred_data(fiscal_quarter):
    # Check if the data for this quarter is already cached
    if fiscal_quarter in cache:
        return cache[fiscal_quarter]
    
    # Extract year and quarter from fiscal quarter format (e.g., '2021Q1')
    year, quarter = fiscal_quarter.split('Q')
    year = int(year)
    quarter = int(quarter)

    # Map quarters to the quarter-end dates (e.g., '2021-03-31' for Q1)
    quarter_end_months = {1: '03', 2: '06', 3: '09', 4: '12'}
    quarter_end = f"{year}-{quarter_end_months[quarter]}-31"

    # Fetch all relevant data for this quarter
    try:
        unemployment_data = fred.get_series('UNRATE').resample('Q').mean()
        gdp_growth_data = fred.get_series('A191RL1Q225SBEA').resample('Q').mean()
        cpi_data = fred.get_series('CPIAUCSL').resample('Q').mean()
        fed_funds_data = fred.get_series('FEDFUNDS').resample('Q').mean()
        consumer_sentiment_data = fred.get_series('UMCSENT').resample('Q').mean()
        investment_data = fred.get_series('PNFIC1').resample('Q').mean()
        labor_participation_data = fred.get_series('CIVPART').resample('Q').mean()
        industrial_prod_data = fred.get_series('INDPRO').resample('Q').mean()
        housing_starts_data = fred.get_series('HOUST').resample('Q').mean()
        corporate_profits_data = fred.get_series('CP').resample('Q').mean()
        disposable_income_data = fred.get_series('DSPIC96').resample('Q').mean()

        # Fetch the values for each quarter based on the quarter-end date
        unemployment_rate = unemployment_data.get(quarter_end, None)
        gdp_growth_rate = gdp_growth_data.get(quarter_end, None)
        inflation_rate = cpi_data.get(quarter_end, None)
        fed_funds_rate = fed_funds_data.get(quarter_end, None)
        consumer_sentiment = consumer_sentiment_data.get(quarter_end, None)
        private_investment = investment_data.get(quarter_end, None)
        labor_participation = labor_participation_data.get(quarter_end, None)
        industrial_production = industrial_prod_data.get(quarter_end, None)
        housing_starts = housing_starts_data.get(quarter_end, None)
        corporate_profits = corporate_profits_data.get(quarter_end, None)
        disposable_income = disposable_income_data.get(quarter_end, None)

        # Store fetched data in the cache
        cache[fiscal_quarter] = pd.Series({
            'Unemployment Rate': unemployment_rate,
            'GDP Growth Rate': gdp_growth_rate,
            'Inflation Rate (CPI)': inflation_rate,
            'Federal Funds Rate': fed_funds_rate,
            'Consumer Sentiment Index': consumer_sentiment,
            'Private Nonresidential Investment': private_investment,
            'Labor Force Participation Rate': labor_participation,
            'Industrial Production Index': industrial_production,
            'Housing Starts': housing_starts,
            'Corporate Profits': corporate_profits,
            'Real Disposable Personal Income': disposable_income
        })

        return cache[fiscal_quarter]

    except Exception as e:
        print(f"Error fetching data for {fiscal_quarter}: {e}")
        return pd.Series({
            'Unemployment Rate': None,
            'GDP Growth Rate': None,
            'Inflation Rate (CPI)': None,
            'Federal Funds Rate': None,
            'Consumer Sentiment Index': None,
            'Private Nonresidential Investment': None,
            'Labor Force Participation Rate': None,
            'Industrial Production Index': None,
            'Housing Starts': None,
            'Corporate Profits': None,
            'Real Disposable Personal Income': None
        })

# Apply the function to each unique fiscal quarter in the dataframe
unique_fiscal_quarters = wfinancial['Fiscal Quarter'].unique()

# Create a dataframe to hold all the macroeconomic data for each quarter
macroeconomic_data = pd.DataFrame([fetch_fred_data(q) for q in unique_fiscal_quarters], index=unique_fiscal_quarters)

/var/folders/_y/b_vc_rg16xdb3hx0mvp678hw0000gn/T/ipykernel_93048/1787984514.py:27: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  unemployment_data = fred.get_series('UNRATE').resample('Q').mean()
/var/folders/_y/b_vc_rg16xdb3hx0mvp678hw0000gn/T/ipykernel_93048/1787984514.py:28: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  gdp_growth_data = fred.get_series('A191RL1Q225SBEA').resample('Q').mean()
/var/folders/_y/b_vc_rg16xdb3hx0mvp678hw0000gn/T/ipykernel_93048/1787984514.py:29: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  cpi_data = fred.get_series('CPIAUCSL').resample('Q').mean()


KeyboardInterrupt: 

In [14]:
print(macroeconomic_data.describe())

       Unemployment Rate  GDP Growth Rate  Inflation Rate (CPI)  \
count          21.000000        21.000000             21.000000   
mean            6.590476         1.190476            237.267714   
std             2.073782         3.841211             21.337708   
min             3.800000        -8.500000            212.377667   
25%             4.900000        -0.900000            219.699000   
50%             6.233333         2.000000            235.355000   
75%             8.266667         3.600000            243.838667   
max             9.933333         7.400000            284.826000   

       Federal Funds Rate  Consumer Sentiment Index  \
count           21.000000                 21.000000   
mean             0.395714                 78.334921   
std              0.697104                 12.652481   
min              0.073333                 57.666667   
25%              0.103333                 70.166667   
50%              0.156667                 75.500000   
75%        

In [16]:
import time
# List of missing fiscal quarters that need data
missing_fiscal_quarters = [
    '2013Q1', '2013Q2', '2013Q3', '2013Q4',
    '2014Q1', '2014Q2', '2014Q3',
    '2017Q2', '2017Q3', '2017Q4',
    '2018Q1', '2018Q2', '2018Q3', '2018Q4',
    '2019Q1', '2019Q2', '2019Q3', '2019Q4',
    '2022Q3', '2022Q4', '2023Q1', '2023Q2', '2023Q3', '2023Q4'
]

# Cache for storing fetched data by fiscal quarter
cache = {}

# Function to fetch macroeconomic data for a given fiscal quarter
def fetch_fred_data(fiscal_quarter):
    if fiscal_quarter in cache:
        return cache[fiscal_quarter]
    
    # Extract year and quarter from fiscal quarter format
    year, quarter = fiscal_quarter.split('Q')
    year = int(year)
    quarter = int(quarter)
    
    # Map quarter to the quarter-end date
    quarter_end_months = {1: '03', 2: '06', 3: '09', 4: '12'}
    quarter_end = f"{year}-{quarter_end_months[quarter]}-31"

    try:
        # Fetch all relevant data for this quarter
        unemployment_data = fred.get_series('UNRATE').resample('Q').mean()
        gdp_growth_data = fred.get_series('A191RL1Q225SBEA').resample('Q').mean()
        cpi_data = fred.get_series('CPIAUCSL').resample('Q').mean()
        fed_funds_data = fred.get_series('FEDFUNDS').resample('Q').mean()
        consumer_sentiment_data = fred.get_series('UMCSENT').resample('Q').mean()
        investment_data = fred.get_series('PNFIC1').resample('Q').mean()
        labor_participation_data = fred.get_series('CIVPART').resample('Q').mean()
        industrial_prod_data = fred.get_series('INDPRO').resample('Q').mean()
        housing_starts_data = fred.get_series('HOUST').resample('Q').mean()
        corporate_profits_data = fred.get_series('CP').resample('Q').mean()
        disposable_income_data = fred.get_series('DSPIC96').resample('Q').mean()

        # Fetch the values for each quarter based on the quarter-end date
        unemployment_rate = unemployment_data.get(quarter_end, None)
        gdp_growth_rate = gdp_growth_data.get(quarter_end, None)
        inflation_rate = cpi_data.get(quarter_end, None)
        fed_funds_rate = fed_funds_data.get(quarter_end, None)
        consumer_sentiment = consumer_sentiment_data.get(quarter_end, None)
        private_investment = investment_data.get(quarter_end, None)
        labor_participation = labor_participation_data.get(quarter_end, None)
        industrial_production = industrial_prod_data.get(quarter_end, None)
        housing_starts = housing_starts_data.get(quarter_end, None)
        corporate_profits = corporate_profits_data.get(quarter_end, None)
        disposable_income = disposable_income_data.get(quarter_end, None)

        # Store fetched data in the cache
        cache[fiscal_quarter] = pd.Series({
            'Unemployment Rate': unemployment_rate,
            'GDP Growth Rate': gdp_growth_rate,
            'Inflation Rate (CPI)': inflation_rate,
            'Federal Funds Rate': fed_funds_rate,
            'Consumer Sentiment Index': consumer_sentiment,
            'Private Nonresidential Investment': private_investment,
            'Labor Force Participation Rate': labor_participation,
            'Industrial Production Index': industrial_production,
            'Housing Starts': housing_starts,
            'Corporate Profits': corporate_profits,
            'Real Disposable Personal Income': disposable_income
        })

        return cache[fiscal_quarter]

    except Exception as e:
        print(f"Error fetching data for {fiscal_quarter}: {e}")
        return pd.Series({
            'Unemployment Rate': None,
            'GDP Growth Rate': None,
            'Inflation Rate (CPI)': None,
            'Federal Funds Rate': None,
            'Consumer Sentiment Index': None,
            'Private Nonresidential Investment': None,
            'Labor Force Participation Rate': None,
            'Industrial Production Index': None,
            'Housing Starts': None,
            'Corporate Profits': None,
            'Real Disposable Personal Income': None
        })

# Fetch missing data and introduce delays to avoid rate limits
for fiscal_quarter in missing_fiscal_quarters:
    print(f"Fetching data for {fiscal_quarter}")
    fetch_fred_data(fiscal_quarter)
    time.sleep(5)  # Introduce a 5-second delay between requests to avoid rate limits

# Merge the missing data back into your dataframe
missing_fiscal_quarters_df = pd.DataFrame([cache[q] for q in missing_fiscal_quarters], index=missing_fiscal_quarters)
print(missing_fiscal_quarters_df)

Fetching data for 2013Q1


/var/folders/_y/b_vc_rg16xdb3hx0mvp678hw0000gn/T/ipykernel_93048/363626593.py:31: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  unemployment_data = fred.get_series('UNRATE').resample('Q').mean()
/var/folders/_y/b_vc_rg16xdb3hx0mvp678hw0000gn/T/ipykernel_93048/363626593.py:32: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  gdp_growth_data = fred.get_series('A191RL1Q225SBEA').resample('Q').mean()
/var/folders/_y/b_vc_rg16xdb3hx0mvp678hw0000gn/T/ipykernel_93048/363626593.py:33: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  cpi_data = fred.get_series('CPIAUCSL').resample('Q').mean()
/var/folders/_y/b_vc_rg16xdb3hx0mvp678hw0000gn/T/ipykernel_93048/363626593.py:34: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  fed_funds_data = fred.get_series('FEDFUNDS').resample('Q').mean()
/

Fetching data for 2013Q2
Fetching data for 2013Q3
Fetching data for 2013Q4
Fetching data for 2014Q1
Fetching data for 2014Q2
Fetching data for 2014Q3
Fetching data for 2017Q2
Fetching data for 2017Q3
Fetching data for 2017Q4
Fetching data for 2018Q1
Fetching data for 2018Q2
Fetching data for 2018Q3
Fetching data for 2018Q4
Fetching data for 2019Q1
Fetching data for 2019Q2
Fetching data for 2019Q3
Fetching data for 2019Q4
Fetching data for 2022Q3
Fetching data for 2022Q4
Fetching data for 2023Q1
Fetching data for 2023Q2
Fetching data for 2023Q3
Fetching data for 2023Q4
        Unemployment Rate  GDP Growth Rate  Inflation Rate (CPI)  \
2013Q1           7.733333              4.0            232.299333   
2013Q2                NaN              NaN                   NaN   
2013Q3                NaN              NaN                   NaN   
2013Q4           6.933333              3.5            234.162667   
2014Q1           6.666667             -1.4            235.621000   
2014Q2           

## Combine All and Export the Dataset

In this part, the final dataset is exported as wfinancial_wfred.csv.

In [25]:
combined_data = pd.concat([macroeconomic_data, missing_fiscal_quarters_df])
combined_data.index.name = 'Fiscal Quarter'

# Drop old macroeconomic columns from wfinancial before merging
macroeconomic_columns = [
    'Unemployment Rate', 'GDP Growth Rate', 'Inflation Rate (CPI)', 'Federal Funds Rate',
    'Consumer Sentiment Index', 'Private Nonresidential Investment', 'Labor Force Participation Rate',
    'Industrial Production Index', 'Housing Starts', 'Corporate Profits', 'Real Disposable Personal Income'
]

# Drop these columns from wfinancial if they exist
wfinancial_cleaned = wfinancial.drop(columns=[col for col in macroeconomic_columns if col in wfinancial.columns])

# Now proceed with the merge as before
wfinancial_updated = wfinancial_cleaned.merge(combined_data, on='Fiscal Quarter', how='left')



In [27]:
output_file = os.path.join(output_dir, 'wfinancial_wfred.csv')

# Export the dataframe to CSV inside the 'Data' folder
wfinancial_updated.to_csv(output_file, index=False)  # index=False to exclude the row numbers

# Print the output path to confirm
print(f"DataFrame exported to: {output_file}")

DataFrame exported to: Data/wfinancial_wfred.csv
